In [1]:
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format

import sys
sys.path.append('../lib/')
from financelib import FinLoad
from financelib import FinCalc
from financelib import FinPlot
from pathlib import Path

In [2]:
YEAR = 2024
data_path_o = Path("../../tmp/data")

### Load Data 2024

In [3]:
init_holdings = FinLoad.load_init_holdings(data_path_o, YEAR)
df_year_cashflow = FinLoad.load_cashflow(data_path_o, YEAR)
df_year_investments = FinLoad.load_investments(data_path_o, YEAR)

### Calculate

In [4]:
df_m_cashflow = FinCalc.calc_monthly_cashflow(df_year_cashflow)

In [5]:
df_m_cashflow.set_index("Date").T

Date,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31
incomes,"2,031.00","1,642.00","1,630.00","1,642.00","1,682.00","1,688.00","2,534.34","1,694.00","1,694.00","1,778.00","1,693.00","2,757.00"
liabilities,"-1,069.21",-878.36,-359.75,-936.96,-614.41,-842.75,-386.74,-389.54,-621.00,-569.47,-544.63,-478.20
savings,961.79,763.64,"1,270.25",705.04,"1,067.59",845.25,"2,147.60","1,304.46","1,073.00","1,208.53","1,148.37","2,278.80"
saving_rate,0.47,0.47,0.78,0.43,0.63,0.50,0.85,0.71,0.63,0.68,0.68,0.83


In [6]:
df_m_cashflow

,Date,incomes,liabilities,savings,saving_rate
0,2024-01-31,"2,031.00","-1,069.21",961.79,0.47
1,2024-02-29,"1,642.00",-878.36,763.64,0.47
2,2024-03-31,"1,630.00",-359.75,"1,270.25",0.78
3,2024-04-30,"1,642.00",-936.96,705.04,0.43
4,2024-05-31,"1,682.00",-614.41,"1,067.59",0.63
5,2024-06-30,"1,688.00",-842.75,845.25,0.50
6,2024-07-31,"2,534.34",-386.74,"2,147.60",0.85
7,2024-08-31,"1,694.00",-389.54,"1,304.46",0.71
8,2024-09-30,"1,694.00",-621.00,"1,073.00",0.63
9,2024-10-31,"1,778.00",-569.47,"1,208.53",0.68


In [7]:
fig_cashflow = FinPlot.plot_cashflow(df_m_cashflow)
fig_cashflow.show()

In [8]:
# Liquidity in Hype, Revolut
current_balance_revolut = round(df_year_cashflow.loc[df_year_cashflow["Type"] == "Revolut"]['Qty'].sum(),2) + init_holdings['liquidity']['Revolut - EUR']
current_balance_hype    = round(df_year_cashflow.loc[df_year_cashflow["Type"] == "Hype"]['Qty'].sum(),2) + init_holdings['liquidity']['Hype - EUR']
current_balance_cash    = round(df_year_cashflow.loc[df_year_cashflow["Type"] == "Cash"]['Qty'].sum(),2) + init_holdings['liquidity']['Money - EUR']
current_balance_bitget  = round(df_year_cashflow.loc[df_year_cashflow["Type"] == "Bitget"]['Qty'].sum(),2)

In [9]:
print("Current LIQUIDITY")
print(f"Revolut: {current_balance_revolut}")
print(f"Hype: {current_balance_hype}")
print(f"Cash: {current_balance_cash}")
print(f"Bitget Fiat: {current_balance_bitget}")

Current LIQUIDITY
Revolut: 28.61
Hype: 14382.58
Cash: 300.0
Bitget Fiat: 0.0


### Detail Expenses January 2025

In [ ]:
df_expenses = FinCalc.calc_expenses(df_year_cashflow[ df_year_cashflow.index.month == 1 ])
fig = FinPlot.plot_expenses_donut(df_expenses)
fig.show()

In [68]:
def plot_expenses_donut(df_expenses, plot_categories = False):
    pxfig = px.sunburst(df_expenses, path=['Category', 'Subcategory'], values='Expenses')
    
    labels = pxfig['data'][0]['labels'].tolist()
    parents = pxfig['data'][0]['parents'].tolist()
    ids = pxfig['data'][0]['ids'].tolist()
    if plot_categories:
        values = None
    else:
        values = pxfig['data'][0]['values'].tolist()
  
    print(parents)
    print(labels)

    # Define colors for each category
    category_colors = {
        'Shop':      '#CDC1FF',
        'Groceries': '#C96868',
        'Other':     '#95D2B3',
        'Leisure':   '#FCDC94',
        'Transport': '#B9B28A',
        'Subs':      '#C9E9D2',
        'Health':    '#D4F6FF',
        'Family':    '#FFCF9D',
        'Holiday':   '#FEFBD8',
        'Bills':     '#E7D4B5'
    }

    # Create a list of colors for each label
    #colors = []
    #for label in labels:
    #    if label in category_colors:
    #        colors.append(category_colors[label])  # Category color
    #    else:
    #        # Get the parent category color and lighten it for subcategories
    #        parent_index = labels.index(parents[labels.index(label)]) if parents[labels.index(label)] else None
    #        if parent_index is not None:
    #            parent_color = category_colors[parents[labels.index(label)]]
    #            colors.append(parent_color)  # Lighter subcategory color
    #        else:
    #            colors.append('#FFFFFF')  # Default color for root

    # Create a list of colors for each label
    colors = []
    for label in labels:
        if label in category_colors:
            colors.append(category_colors[label])  # Category color
        else:
            if parents[labels.index(label)]:
                parent_index = labels.index(parents[labels.index(label)])
            else:
                parent_index = None

            if parent_index is not None:
                parent_color = category_colors[parents[labels.index(label)]]
                colors.append(parent_color)  # Lighter subcategory color
            else:
                colors.append('#FFFFFF')  # Default color for root

    fig = go.Figure(
        go.Sunburst(
            labels = labels,
            parents = parents,
            values = values,
            ids = ids,
            branchvalues = "total",
            marker=dict(colors=colors)
        )
    )
    fig.update_layout(
        title = dict(text="Expenses", x=0.5, y=0.95),
        margin = dict(t=60, l=10, r=10, b=10),
        height = 500, width = 500
    )
    return fig

In [69]:
df_expenses = FinCalc.calc_expenses(df_year_cashflow[ df_year_cashflow.index.month == 8 ])
fig = plot_expenses_donut(df_expenses)
fig.show()

['Shop', 'Subs', 'Groceries', 'Leisure', 'Family', 'Groceries', 'Groceries', 'Transport', 'Holiday', 'Holiday', 'Leisure', 'Leisure', 'Subs', 'Groceries', 'Holiday', 'Subs', '', '', '', '', '', '', '']
['Accessories', 'Bank', 'Diet', 'Drink', 'Favors', 'Feast', 'Food', 'Gasoline', 'Groceries', 'Internet', 'Karma', 'Restaurant', 'SIM', 'Supplements', 'Transport', 'VPS', 'Family', 'Groceries', 'Holiday', 'Leisure', 'Shop', 'Subs', 'Transport']


### Investments 2024

In [5]:
symbols = list(df_year_investments.loc[df_year_investments["Type"]=="Cryptocurrencies"]["Symbol"].unique())
symbols

['SOL', 'ETH', 'USDT']

In [6]:
# BUILD THE DATAFRAME OF ASSET MOVEMENTS WITH FILL TILL END OF YEAR
list_investments_asset_class = dict()
for asset_class in ["Cryptocurrencies"]:
    df_year_investments_per_asset = dict()
    for symbol in symbols:
        df_symbol_transactions = df_year_investments.loc[df_year_investments["Type"] == asset_class].loc[df_year_investments['Symbol'] == symbol]
        df_month_invest = df_symbol_transactions.resample(rule='ME')['Qty'].sum()
        print(df_month_invest)

        try:
            fill_len = 12 - len(df_month_invest)
            df_month_fill = pd.DataFrame(index = pd.date_range(f'{year}-{len(df_month_invest)+1:0=2}-01',f'{year}-12-31', freq='ME'), data = zip([0]*fill_len), columns=["Qty"])
            df_year_investments_per_asset[symbol] = pd.concat([df_month_invest, df_month_fill])
        except:
            df_year_investments_per_asset[symbol] = df_month_invest
    
    list_investments_asset_class[asset_class] = df_year_investments_per_asset

Date
2024-01-31   0.00
Freq: ME, Name: Qty, dtype: float64
Date
2024-01-31   0.00
Freq: ME, Name: Qty, dtype: float64
Date
2024-01-31     0.00
2024-02-29     0.00
2024-03-31     0.00
2024-04-30     0.00
2024-05-31     0.00
2024-06-30     0.00
2024-07-31     0.00
2024-08-31   108.24
Freq: ME, Name: Qty, dtype: float64


In [10]:
def getSymbolHoldings(asset_class, asset_ticker, list_investments_asset_class, init_holdings, year):
    init_and_cum_qty_permonth = list_investments_asset_class[asset_class][asset_ticker].values.cumsum() + init_holdings[asset_class][asset_ticker]
    asset_history = pd.read_csv(f"{data_path_o}/exchange/{asset_ticker}-EUR.csv").set_index('Date')
    asset_history["shift"] = asset_history.shift(1)["Close €"]
    asset_history["Returns"] = (asset_history["Close €"] - asset_history["shift"] )/ asset_history["Close €"]
    pick_symbol_holdings = asset_history.loc[f'{year}-01-01':f'{year}-12-31'].drop(columns=["shift"])
    
    pick_symbol_holdings["shares"] = init_and_cum_qty_permonth
    pick_symbol_holdings["holdings €"] = pick_symbol_holdings["Close €"]*pick_symbol_holdings["shares"]

    return pick_symbol_holdings

In [11]:
getSymbolHoldings("Cryptocurrencies", "SOL", list_investments_asset_class, init_holdings, YEAR).T

ValueError: Length of values (1) does not match length of index (12)

In [32]:
getSymbolHoldings("Cryptocurrencies", "ETH", list_investments_asset_class, init_holdings, year).T

Date,2024-01-30,2024-02-28,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31
Close €,"2,113.56","3,085.93","3,379.90","2,823.98","3,467.94","3,200.17","2,987.01","2,275.55","2,336.43","2,311.38","3,502.00","3,221.44"
Returns,0.02,0.32,0.09,-0.20,0.19,-0.08,-0.07,-0.31,0.03,-0.01,0.34,-0.09
shares,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
holdings €,25.79,37.65,41.23,34.45,42.31,39.04,36.44,27.76,28.50,28.20,42.72,39.30


In [33]:
getSymbolHoldings("Cryptocurrencies", "USDT", list_investments_asset_class, init_holdings, year).T

Date,2024-01-30,2024-02-28,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31
Close €,0.93,0.92,0.93,0.94,0.92,0.93,0.92,0.91,0.90,0.92,0.95,0.96
Returns,0.02,-0.00,0.00,0.01,-0.02,0.01,-0.01,-0.02,-0.01,0.02,0.03,0.02
shares,0.00,0.00,0.00,0.00,0.00,0.00,0.00,108.24,108.24,108.24,108.24,108.24
holdings €,0.00,0.00,0.00,0.00,0.00,0.00,0.00,97.99,97.17,99.33,102.33,104.28


In [34]:
cryptocurrency_year_holdings = 0
for symbol in symbols:
    df_symbol_holdings = getSymbolHoldings("Cryptocurrencies", symbol, list_investments_asset_class, init_holdings, year)
    cryptocurrency_year_holdings += df_symbol_holdings["holdings €"]

In [35]:
cryptocurrency_year_holdings

Date
2024-01-30     462.44
2024-02-28     601.79
2024-03-31     952.95
2024-04-30     611.48
2024-05-31     783.76
2024-06-30     702.00
2024-07-31     807.59
2024-08-31     721.44
2024-09-30     791.55
2024-10-31     880.07
2024-11-30   1,236.90
2024-12-31   1,030.07
Name: holdings €, dtype: float64

In [36]:
# Then SUM on NW

### NW 2024

In [37]:
init_holdings['liquidity']

{'Hype - EUR': 6, 'Revolut - EUR': 16.74, 'Money - EUR': 14.13}

In [38]:
liquidity = df_m_cashflow["savings"].values.cumsum() + init_holdings['liquidity']['Hype - EUR'] + init_holdings['liquidity']['Revolut - EUR'] + init_holdings['liquidity']['Money - EUR']

In [39]:
list(cryptocurrency_year_holdings.values)

[np.float64(462.44376800000003),
 np.float64(601.7949739999999),
 np.float64(952.9486120000001),
 np.float64(611.479244),
 np.float64(783.7628880000001),
 np.float64(702.000634),
 np.float64(807.592586),
 np.float64(721.4430884000001),
 np.float64(791.5517060000001),
 np.float64(880.0685808000001),
 np.float64(1236.8958344),
 np.float64(1030.0694224)]

In [40]:
liquidity

array([  998.66 ,  1762.3  ,  3032.55 ,  3737.59 ,  4805.18 ,  5650.43 ,
        7798.03 ,  9102.49 , 10175.491, 11384.021, 12532.391, 14811.191])

In [41]:
liquidity_end_2023 = 24.30
crypto_end_2023 = 473.56
columns = ["liquidity", "crypto"]

nw2023 = pd.DataFrame(data = zip([liquidity_end_2023],[crypto_end_2023]), columns = columns, index = pd.date_range(f'2023-12-31',f'2023-12-31', freq='ME'))

In [42]:
zipped = zip(liquidity, cryptocurrency_year_holdings.values)
columns = ["liquidity", "crypto"]
nw2024 = pd.DataFrame(index = pd.date_range(f'{year}-01-01',f'{year}-12-31', freq='ME'), data = zipped, columns = columns)

In [43]:
nw = pd.concat([nw2023, nw2024])

In [44]:
nw["nw"] = nw.liquidity + nw.crypto

In [45]:
nw["nwch"] = (nw["nw"] - nw.nw.shift(1) )
nw["nw%"] = (nw["nw"] - nw.nw.shift(1) )/ nw["nw"]

In [46]:
nw.T

,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31
liquidity,24.30,998.66,"1,762.30","3,032.55","3,737.59","4,805.18","5,650.43","7,798.03","9,102.49","10,175.49","11,384.02","12,532.39","14,811.19"
crypto,473.56,462.44,601.79,952.95,611.48,783.76,702.00,807.59,721.44,791.55,880.07,"1,236.90","1,030.07"
nw,497.86,"1,461.10","2,364.09","3,985.50","4,349.07","5,588.94","6,352.43","8,605.62","9,823.93","10,967.04","12,264.09","13,769.29","15,841.26"
nwch,NaN,963.24,902.99,"1,621.40",363.57,"1,239.87",763.49,"2,253.19","1,218.31","1,143.11","1,297.05","1,505.20","2,071.97"
nw%,NaN,0.66,0.38,0.41,0.08,0.22,0.12,0.26,0.12,0.10,0.11,0.11,0.13


### FIRE Metrics on end 2024

In [47]:
current_yearly_expediture = df_m_cashflow.liabilities.sum()
current_nw = nw.nw.iloc[-1]

In [48]:
desired_wd = 0.030
fu_number = abs(current_yearly_expediture) / desired_wd
print(fu_number)

256367.3


In [49]:
progress_fi = current_nw / fu_number
print(progress_fi)

0.06179126753841071


In [50]:
current_yearly_saved = nw.nwch.sum()

In [51]:
current_yearly_monthly_savings = current_yearly_saved/12

In [52]:
months_left_to_fire = (fu_number - current_nw)/current_yearly_monthly_savings
print(f"Months left to FIRE: {int(months_left_to_fire)}")

Months left to FIRE: 188
